In [2]:
library(lme4)
library(lmerTest)
library(effects)
library(stringr)


Loading required package: Matrix

Attaching package: ‘lmerTest’

The following object is masked from ‘package:lme4’:

    lmer

The following object is masked from ‘package:stats’:

    step

Loading required package: carData
lattice theme set by effectsTheme()
See ?effectsTheme for details.


In [3]:
data_profile <- read.csv('/Users/m221138/RA_project/analysis/linear_model_r1/MLR.clp.sc.ready.ignor.norm.qc.tsv', sep="\t", row.names=1, header=TRUE)
data_profile <- as.data.frame(t(data_profile))
plot_dir <- '/Users/m221138/RA_project/analysis/linear_model_r1/plots/clp/sc'

In [5]:
output_txt = '/Users/m221138/RA_project/analysis/linear_model_r1/MLR.clp.sc.tsv'
if (file.exists(output_txt)){
    file.remove(output_txt)
}

NUM_CHEM_ID <- ncol(data_profile)
NUM_patient_timepoint <- nrow(data_profile)

cat('chemID\tsimpleLRM_Fstatistics\tmultiLRM(chem_sig)\tmultiLRM(sex_sig)\tmultiLRM(age_sig)\n', file=output_txt, append=TRUE)

print (NUM_CHEM_ID)
print (NUM_patient_timepoint)

for (i in 1:NUM_CHEM_ID){
    if (i > 4){
        output_string <- ''
       
        chemID <- colnames(data_profile)[i]
        chem_profile <- data_profile[,i]
        print (chemID)
        
        output_pdf <- paste(plot_dir, '/', i, ".pdf")
        output_pdf <- str_replace_all(output_pdf," ","")
        
        simple_LRM <- lm(DAS ~ chem_profile, data=data_profile)
        simple_LRM_stats <- summary(simple_LRM)$fstatistic
        print (simple_LRM_stats)
        simple_LRM_f_pvalue<- pf(simple_LRM_stats[1], simple_LRM_stats[2], simple_LRM_stats[3], lower.tail=FALSE)
        simple_LRM_rsquare <- summary(simple_LRM)$adj.r.squared
        
        multi_LRM <- lmer(DAS ~ chem_profile + SEX + AGE + (1|patient_dummyID), data=data_profile)
        print (multi_LRM)
        break
        chem_sig <- anova(multi_LRM)$Pr[1]
        sex_sig <- anova(multi_LRM)$Pr[2]
        age_sig <- anova(multi_LRM)$Pr[3]
        
        output_string <- paste(chemID,"\t", simple_LRM_f_pvalue,"\t", chem_sig, "\t", sex_sig, "\t", age_sig, "\n")
        cat(output_string, file=output_txt, append=TRUE)
        
        plot1 <- allEffects(simple_LRM)
        plot2 <- allEffects(multi_LRM)[1]
        plot(allEffects(multi_LRM)[1], main="MLM", xlab=chemID)
        
        if (chem_sig < 0.05){
            #pdf(output_pdf)
            par(mfrow=c(1,2))
            plot(allEffects(simple_LRM), main="SLM")
            plot(allEffects(multi_LRM)[1], main="MLM")
            #dev.off()
        }
        break
    }
}


[1] TRUE

[1] 825
[1] 128
[1] "LPC(20:3)"
       value        numdf        dendf 
5.003227e-03 1.000000e+00 1.260000e+02 
Linear mixed model fit by REML ['lmerModLmerTest']
Formula: DAS ~ chem_profile + SEX + AGE + (1 | patient_dummyID)
   Data: data_profile
REML criterion at convergence: 420.4308
Random effects:
 Groups          Name        Std.Dev.
 patient_dummyID (Intercept) 1.0424  
 Residual                    0.8686  
Number of obs: 128, groups:  patient_dummyID, 64
Fixed Effects:
 (Intercept)  chem_profile           SEX           AGE  
     3.67579      -0.04106      -0.37121      -0.00678  
